<a href="https://colab.research.google.com/github/ParkEunHyeok/Project_Kermit/blob/main/GPT2/Kermit_GPT2_ChatbotTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm
!pip install kobert-transformers==0.4.1
!pip install kogpt2-transformers==0.3.0
!pip install transformers==3.0.2
!pip install torch
!pip install tokenizers==0.8.1rc1
!pip install kss

In [ ]:
# 구글 드라이브 연결
import os
import numpy as np
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from kogpt2_transformers import get_kogpt2_tokenizer, get_kogpt2_model

In [ ]:
class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = get_kogpt2_model()

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 40,
               top_p=0.92,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size =None,
               num_return_sequences=3,
               early_stopping=False,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs

In [ ]:
root_path=path
data_path = f"{root_path}/data/wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20210922-add-chatbotdata.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

In [ ]:
while 1:
#for i in range(5):
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)
  # set top_k to 50
  sample_output = model.generate(
      input_ids=input_ids
      )


  print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')
  print(sample_output.shape)
  print(sample_output[0])
  print(sample_output[1])
  print(sample_output[2])

# for s in kss.split_sentences(sent):
#     print(s)

Question: 안녕하세요


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 안녕하세요.안녕하세요.안녕하세요.안녕하세요.안녕하세요.안녕하세요.안녕하세요.
----------------------------------------------------------------------------------------------------
torch.Size([3, 40])
tensor([    0, 28911,  4455,     1,     0, 28911,  4455, 47440,     1,     0,
        28911,  4455, 47440,     1,     0, 28911,  4455, 47440,     1,     0,
        28911,  4455, 47440,     1,     0, 28911,  4455, 47440,     1,     0,
        28911,  4455, 47440,     1,     0, 28911,  4455, 47440,     1,     0])
tensor([    0, 28911,  4455,     1,     0, 28911,  4455, 47440,     1,     0,
         6627,   250,  3158,   184, 14716,   484, 47440,     1,     0,  2288,
         2909,  1689,   947, 12724, 17290,  2825, 47440,     1,     0,  6323,
         2634, 43253, 48360, 13139, 47440,  5633,   127, 29119,   352,   494])
tensor([    0, 28911,  4455,     1,     0, 28911,  4455, 47440,     1,     0,
         5633, 24978,  4042, 47440,     1,     0, 24978,  4042, 47440,     1,
            0, 12266,  6076, 21820, 47537,  40

KeyboardInterrupt: ignored